In [1]:
import pandas as pd 
import missingno as msno 
import numpy as np 
import seaborn as sms
from datetime import datetime as dt 
import matplotlib.pyplot as plt
import plotly.express as px 
from dash import Dash, dcc, html, Input, Output
import sklearn 
from sklearn.linear_model import LinearRegression, Ridge, Lasso   
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
import pickle 
from collections import OrderedDict



pd.options.display.max_columns = 150 
pd.options.display.max_rows = 150 

## Recuperation du CSV

In [2]:
df_modelisation = pd.read_csv('df2.csv')
df_modelisation_prix = df_modelisation[['price']]

### On supprime les colonnes qui nous semble inutile

In [3]:
df_modelisation = df_modelisation.drop('id', axis=1)
df_modelisation = df_modelisation.drop('date_sale', axis=1)
df_modelisation = df_modelisation.drop('price_log', axis=1)
df_modelisation = df_modelisation.drop('sqft_living15', axis=1)
df_modelisation = df_modelisation.drop('sqft_lot15', axis=1)

### On passe month et zipcode en objet 

In [4]:
data_types_dict = {'month': str} 
df_modelisation = df_modelisation.astype(data_types_dict)  

In [5]:
data_types_dict = {'zipcode': str} 
df_modelisation = df_modelisation.astype(data_types_dict)  

### ????

In [6]:
X = df_modelisation.drop('price', axis=1)
y = df_modelisation[['price']]
X_train, X_test, y_train, y_test = train_test_split(X,y)
num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)
my_num_pipe = make_pipeline(StandardScaler(), PolynomialFeatures())
preprocessing = ColumnTransformer([
    ("one_hot", OneHotEncoder(),cat_col),
    ("scaling", my_num_pipe, num_col)
])


In [7]:
list(num_col)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'year']

In [8]:
list(cat_col)

['zipcode', 'month']

In [9]:
preprocessing = ColumnTransformer(transformers=[('one_hot', OneHotEncoder(), cat_col),
                                ('scaling', StandardScaler(), num_col)])                              

# LINEAR REGRESSION   LR

In [10]:
my_pipe_lr = make_pipeline(preprocessing, LinearRegression())

In [11]:
my_pipe_lr = Pipeline(
    [
        ("preprocessing", preprocessing), 
        ('ridge', LinearRegression())
    ]
)

### Fit - Score - Predict

In [12]:
my_pipe_lr.fit(X_train, y_train)
print ("my_pipe_lr score : " , my_pipe_lr.score(X_test, y_test))
print ("my_pipe_lr predict : " , my_pipe_lr.predict(X_train))

my_pipe_lr score :  0.810672758522142
my_pipe_lr predict :  [[293485.15621695]
 [344353.35212661]
 [144970.6266166 ]
 ...
 [305618.09903957]
 [183320.92638629]
 [552436.08179626]]


In [13]:
y_pred = my_pipe_lr.predict(X_train)
y_pred.reshape(1,16209)
y_pred

array([[293485.15621695],
       [344353.35212661],
       [144970.6266166 ],
       ...,
       [305618.09903957],
       [183320.92638629],
       [552436.08179626]])

In [14]:
y_train

,price
7972,268000.0
20968,324500.0
3286,180000.0
18874,381000.0
3660,337500.0
...,...
7854,1000000.0
3095,210000.0
16829,194000.0
9445,297000.0


In [15]:
residual = (y_pred-y_train).abs().sort_values(by='price', axis= 0 , ascending= False)
residual

,price
7252,4.394292e+06
9254,2.947115e+06
1448,2.565801e+06
1315,2.333656e+06
12370,2.290674e+06
...,...
17467,2.531479e+01
10837,1.986878e+01
13175,1.441123e+01
18029,1.286866e+01


In [16]:
to_drop = residual[residual > 500000].index
df_cleaned = df_modelisation.drop(to_drop)   
df_cleaned = df_cleaned.drop('price', axis=1)
df_cleaned

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,year,month
6,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2014,6
7,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,2015,1
16,3,2.00,1890,14040,2.0,0,0,3,7,1890,0,1994,0,98019,47.7277,-121.962,2014,7
24,3,2.25,2450,6500,2.0,0,0,4,8,2450,0,1985,0,98030,47.3739,-122.172,2014,11
29,4,2.50,2570,7173,2.0,0,0,3,8,2570,0,2005,0,98052,47.7073,-122.110,2015,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21580,3,3.00,2780,6000,2.0,0,0,3,9,2780,0,2013,0,98065,47.5184,-121.886,2014,10
21589,3,2.50,2540,4760,2.0,0,0,3,8,2540,0,2010,0,98038,47.3452,-122.022,2014,9
21596,5,2.75,3600,9437,2.0,0,0,3,9,3600,0,2014,0,98059,47.4822,-122.131,2014,8
21605,4,2.50,2520,6023,2.0,0,0,3,9,2520,0,2014,0,98056,47.5137,-122.167,2014,10


### Fit - Score - Predict

In [17]:
my_pipe_lr.fit(X_train, y_train)
print ("my_pipe_lr score : " , my_pipe_lr.score(X_test, y_test))
print ("my_pipe_lr predict : " , my_pipe_lr.predict(X_train))

my_pipe_lr score :  0.810672758522142
my_pipe_lr predict :  [[293485.15621695]
 [344353.35212661]
 [144970.6266166 ]
 ...
 [305618.09903957]
 [183320.92638629]
 [552436.08179626]]


## Stocker le modèle de régression choisi à l'aide du module pickle 

In [18]:
my_pipe_lr
with open("my_pipe_lr.pkl", "wb") as f:
    pickle.dump(my_pipe_lr, f)

def load_from_pickle(name):
    with open(name, "rb") as f:
        return pickle.load(f)


In [19]:
load_from_pickle("my_pipe_lr.pkl")

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                                  ['zipcode', 'month']),
                                                 ('scaling', StandardScaler(),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'waterfront',
                                                   'view', 'condition', 'grade',
                                                   'sqft_above',
                                                   'sqft_basement', 'yr_built',
                                                   'yr_renovated', 'lat',
                                                   'long', 'year'])])),
                ('ridge', LinearRegression())])

# RIDGE

### Pipeline

In [20]:
my_pipe_ridge = Pipeline(
    [
        ("preprocessing", preprocessing), 
        ('ridge', Ridge())
    ]
)

### Recherche des meilleurs paramètres 

In [21]:
my_pipe_ridge.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                    ['zipcode', 'month']),
                                   ('scaling', StandardScaler(),
                                    ['bedrooms', 'bathrooms', 'sqft_living',
                                     'sqft_lot', 'floors', 'waterfront', 'view',
                                     'condition', 'grade', 'sqft_above',
                                     'sqft_basement', 'yr_built', 'yr_renovated',
                                     'lat', 'long', 'year'])])),
  ('ridge', Ridge())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                  ['zipcode', 'month']),
                                 ('scaling', StandardScaler(),
                                  ['bedrooms', 'bathrooms', 'sqft_living',
                                   'sqft_lot', 'floors', 'waterfront', '

In [22]:
hyperparametres = {'ridge__alpha': [0.01,0.1, 0.2,0.5, 1,10]}

In [23]:
random_search = GridSearchCV(my_pipe_ridge, hyperparametres, cv= 5)
random_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('one_hot',
                                                                         OneHotEncoder(),
                                                                         ['zipcode',
                                                                          'month']),
                                                                        ('scaling',
                                                                         StandardScaler(),
                                                                         ['bedrooms',
                                                                          'bathrooms',
                                                                          'sqft_living',
                                                                          'sqft_lot',
                                                                          'floors',
                                                                          'waterfront',
                                                                          'view',
                                                                          'condition',
                                                                          'grade',
                                                                          'sqft_above',
                                                                          'sqft_basement',
                                                                          'yr_built',
                                                                          'yr_renovated',
                                                                          'lat',
                                                                          'long',
                                                                          'year'])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.01, 0.1, 0.2, 0.5, 1, 10]})

In [24]:
random_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('one_hot',
                                                                         OneHotEncoder(),
                                                                         ['zipcode',
                                                                          'month']),
                                                                        ('scaling',
                                                                         StandardScaler(),
                                                                         ['bedrooms',
                                                                          'bathrooms',
                                                                          'sqft_living',
                                                                          'sqft_lot',
                                                                          'floors',
                                                                          'waterfront',
                                                                          'view',
                                                                          'condition',
                                                                          'grade',
                                                                          'sqft_above',
                                                                          'sqft_basement',
                                                                          'yr_built',
                                                                          'yr_renovated',
                                                                          'lat',
                                                                          'long',
                                                                          'year'])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.01, 0.1, 0.2, 0.5, 1, 10]})

In [25]:
random_search.best_params_

{'ridge__alpha': 1}

In [26]:
my_pipe_ridge.set_params(**random_search.best_params_)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                                  ['zipcode', 'month']),
                                                 ('scaling', StandardScaler(),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'waterfront',
                                                   'view', 'condition', 'grade',
                                                   'sqft_above',
                                                   'sqft_basement', 'yr_built',
                                                   'yr_renovated', 'lat',
                                                   'long', 'year'])])),
                ('ridge', Ridge(alpha=1))])

### Fit - Score - Predict

In [27]:
my_pipe_ridge.fit(X_train, y_train)
print ("my_pipe_ridge score : " , my_pipe_ridge.score(X_test, y_test))
print ("my_pipe_ridge predict : " , my_pipe_ridge.predict(X_train))

my_pipe_ridge score :  0.8111474725184868
my_pipe_ridge predict :  [[288080.32304623]
 [334685.3720334 ]
 [131378.05281038]
 ...
 [303994.15449403]
 [180076.90412131]
 [556335.2316358 ]]


# LASSO

### Pipeline

In [28]:
my_pipe_lasso = make_pipeline(preprocessing, Lasso())

In [29]:
my_pipe_lasso = Pipeline(
    [
        ("preprocessing", preprocessing), 
        ('lasso', Lasso())
    ]
)

### Fit - Score - Predict

In [30]:
my_pipe_lasso.fit(X_train, y_train)
print ("my_pipe_lasso score : " , my_pipe_lasso.score(X_test, y_test))
print ("my_pipe_lasso predict : " , my_pipe_lasso.predict(X_train))

my_pipe_lasso score :  0.8106841368436568
my_pipe_lasso predict :  [293393.03689571 344177.41121119 144602.70643435 ... 305612.98280818
 183816.06511538 552733.30399133]


/home/kahoul/miniconda3/envs/ykp/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 43783802287266.47, tolerance: 217861634601.12674
  model = cd_fast.sparse_enet_coordinate_descent(


# ELASTIC NET 

### Pipeline

In [31]:
my_pipe_elastic_net = make_pipeline(preprocessing, ElasticNet())

In [32]:
my_pipe_elastic_net = Pipeline(
    [
        ("preprocessing", preprocessing), 
        ('elastic_net', ElasticNet())
    ]
)

### Fit - Score - Predict

In [33]:
my_pipe_elastic_net.fit(X_train, y_train)
print ("my_pipe_elastic_net score : " , my_pipe_elastic_net.score(X_test, y_test))
print ("my_pipe_elastic_net predict : " , my_pipe_elastic_net.predict(X_train))

my_pipe_elastic_net score :  0.6715724196698876
my_pipe_elastic_net predict :  [334885.33396154 373212.77343284 205692.70866269 ... 268233.73517639
 358875.41840127 636336.07628848]


## Comparaison des modèles

In [34]:
modeles = { 
"my_pipe_lr.score" : my_pipe_lr.score(X_test, y_test), "my_pipe_elastic_net.score ": my_pipe_elastic_net.score(X_test, y_test),
"my_pipe_lasso.score" : my_pipe_lasso.score(X_test, y_test), "my_pipe_ridge.score" : my_pipe_ridge.score(X_test, y_test)
}

modeles = sorted(modeles.items(), key=lambda x: x[1])
modeles

[('my_pipe_elastic_net.score ', 0.6715724196698876),
 ('my_pipe_lr.score', 0.810672758522142),
 ('my_pipe_lasso.score', 0.8106841368436568),
 ('my_pipe_ridge.score', 0.8111474725184868)]

# SUPPRIMER LES VALEURS INFLUENTES

In [35]:
df_cleaned.to_csv("df_cleaned.csv")  

In [36]:
df_modelisation_price = df_modelisation["price"]
df_modelisation_price.to_csv("df_modelisation_price.csv")

## Modélisation de la méthode de régression linéaire

In [37]:
""" plt.scatter(X, y)
plt.plot(X, model.predict(X), c='red') """

" plt.scatter(X, y)\nplt.plot(X, model.predict(X), c='red') "

In [38]:
""" plt.plot(df_modelisation2 , df_modelisation_prix,'ro', markersize = 4) #UTILISATION DE LA FONCTION PLOT 
plt.show() """


" plt.plot(df_modelisation2 , df_modelisation_prix,'ro', markersize = 4) #UTILISATION DE LA FONCTION PLOT \nplt.show() "

In [39]:
""" predictions = model.predict(X_test)
plt.scatter(X_test,y)
plt.plot(X, predictions, c='r') """

" predictions = model.predict(X_test)\nplt.scatter(X_test,y)\nplt.plot(X, predictions, c='r') "

## Fonction qui permet de prédire le prix d'une maison

In [40]:
""" def prediction_maison(model,sqft_living	,grade	,sqft_above	,sqft_living15	,bathrooms,	view	,sqft_basement	,
                bedrooms	,zipcode_98004	,waterfront	,floors	,zipcode_98039,	zipcode_98040	,zipcode_98112,	zipcode_98006,	yr_renovated	,
                zipcode_98033,	zipcode_98105	,sqft_lot,	zipcode_98075,	zipcode_98199	,
                sqft_lot15 ,	zipcode_98001,	zipcode_98042,	zipcode_98023):
    x = np.array([sqft_living	,grade	,sqft_above	,sqft_living15	,bathrooms,	view	,sqft_basement	,
                bedrooms	,zipcode_98004	,waterfront	,floors	,zipcode_98039,	zipcode_98040	,zipcode_98112,	zipcode_98006,	yr_renovated	,
                zipcode_98033,	zipcode_98105	,sqft_lot,	zipcode_98075,	zipcode_98199	,
                sqft_lot15 ,	zipcode_98001,	zipcode_98042,	zipcode_98023]).reshape(1,25)
    print (model.predict(x)) """


' def prediction_maison(model,sqft_living\t,grade\t,sqft_above\t,sqft_living15\t,bathrooms,\tview\t,sqft_basement\t,\n                bedrooms\t,zipcode_98004\t,waterfront\t,floors\t,zipcode_98039,\tzipcode_98040\t,zipcode_98112,\tzipcode_98006,\tyr_renovated\t,\n                zipcode_98033,\tzipcode_98105\t,sqft_lot,\tzipcode_98075,\tzipcode_98199\t,\n                sqft_lot15 ,\tzipcode_98001,\tzipcode_98042,\tzipcode_98023):\n    x = np.array([sqft_living\t,grade\t,sqft_above\t,sqft_living15\t,bathrooms,\tview\t,sqft_basement\t,\n                bedrooms\t,zipcode_98004\t,waterfront\t,floors\t,zipcode_98039,\tzipcode_98040\t,zipcode_98112,\tzipcode_98006,\tyr_renovated\t,\n                zipcode_98033,\tzipcode_98105\t,sqft_lot,\tzipcode_98075,\tzipcode_98199\t,\n                sqft_lot15 ,\tzipcode_98001,\tzipcode_98042,\tzipcode_98023]).reshape(1,25)\n    print (model.predict(x)) '

In [41]:
""" #essai de la fonction
prediction_maison(model,50,7,30,48,2,2,0,2,1,0,1,0,0,0,0,1988,0,0,35,0,0,40,0,0,0) """

' #essai de la fonction\nprediction_maison(model,50,7,30,48,2,2,0,2,1,0,1,0,0,0,0,1988,0,0,35,0,0,40,0,0,0) '